# Description of the project.

As an analyst at a big online store, together with the marketing department, we look at the list of hypotheses that may help boost revenue. We look at the results of A/B hypothesis testing. Our goal is to analyze the results and decide whether it is reasonable to continue testing. 

## Data used in the first part of the project:
### /datasets/hypotheses_us.csv 


Hypotheses — brief descriptions of the hypotheses

Reach — user reach, on a scale of one to ten

Impact — impact on users, on a scale of one to ten

Confidence — confidence in the hypothesis, on a scale of one to ten

Effort — the resources required to test a hypothesis, on a scale of one to ten. The higher the Effort value, the more resource-intensive the test.


## Data used in the second part of the project:
### /datasets/orders_us.csv 



transactionId — order identifier

visitorId — identifier of the user who placed the order

date — of the order

revenue — from the order

group — the A/B test group that the user belongs to




### /datasets/visits_us.csv 

date — date

group — A/B test group

visits — the number of visits on the date specified in the A/B test group specified

In [ ]:
import pandas as pd
import numpy as np
from functools import reduce
from scipy import stats as st
from numpy import random
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()


In [ ]:
hypotheses = pd.read_csv('/datasets/hypotheses_us.csv', sep=';')
orders = pd.read_csv('/datasets/orders_us.csv')
visits = pd.read_csv('/datasets/visits_us.csv')

display(hypotheses.info())

display(hypotheses.head(10))
display(orders.info())
display(visits.info())

Below I am removing the orders that got mistakingly assigned to both groups. It's a pretty high percentage of orders; however, we have to remove them, since there is no way to know, which group is correct and it can seriously skew the graphs. 

In [ ]:
dupl = orders.groupby('visitorId').agg({'group':'nunique'})
dupl_ = dupl[dupl['group']>1].reset_index()
orders = orders.loc[~orders['visitorId'].isin(dupl_['visitorId'])]

Part 1. Prioritizing Hypotheses.

In [ ]:

#ICE
hypotheses['ICE'] = hypotheses['Impact'] * hypotheses['Confidence'] /hypotheses['Effort']

print(hypotheses[['Hypothesis','ICE']].sort_values(by='ICE', ascending=False))




In [ ]:
#RICE
hypotheses['RICE'] = hypotheses['Reach'] * hypotheses['Impact'] * hypotheses['Confidence'] /hypotheses['Effort']
print(hypotheses[['Hypothesis','RICE']].sort_values(by='RICE', ascending=False))

Part 2. A/B Test Analysis.

Graph cumulative revenue by group.

In [ ]:
orders['date'] = pd.to_datetime(orders['date'], format="%Y-%m-%d %H:%M")
visits['date'] = pd.to_datetime(visits['date'], format="%Y-%m-%d %H:%M")


In [ ]:

datesGroups = orders[['date','group']].drop_duplicates() 
ordersAggregated = datesGroups.apply(
    lambda x: orders[np.logical_and(orders['date'] <= x['date'], orders['group'] == x['group'])].agg({
'date' : 'max',
'group' : 'max',
'transactionId' : pd.Series.nunique,
'visitorId' : pd.Series.nunique,
'revenue' : 'sum'}), axis=1).sort_values(by=['date','group']) 



In [ ]:
visitorsAggregated = datesGroups.apply(lambda x: visits[np.logical_and(visits['date'] <= x['date'], visits['group'] == x['group'])].agg({'date' : 'max', 'group' : 'max', 'visits' : 'sum'}), axis=1).sort_values(by=['date','group']) 

In [ ]:
cumulativeData = ordersAggregated.merge(visitorsAggregated, left_on=['date', 'group'], right_on=['date', 'group'])
cumulativeData.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']

In [ ]:
cumulativeRevenueA = cumulativeData[cumulativeData['group']=='A'][['date','revenue', 'orders']]
cumulativeRevenueB = cumulativeData[cumulativeData['group']=='B'][['date','revenue', 'orders']]

In [ ]:
plt.plot(cumulativeRevenueA['date'], cumulativeRevenueA['revenue'], label='A')
plt.plot(cumulativeRevenueB['date'], cumulativeRevenueB['revenue'], label='B')
plt.xticks(rotation=90)
plt.title('Cumulative Revenue')
plt.xlabel('Date')
plt.ylabel('Revenue')
plt.legend()

We can see from this graph that values for group B ar slightly lower in the beginning of the month, but very quickly become mych higher with very high growth between 2019-08-17 and 21. We can assume there was huge sudden growth between these days and after that revenue kept steadily climbing up with group B a clear leader.

Graph cumulative average order size by group.
Cumulative graphs (by day) for average purchase size:

In [ ]:


plt.plot(cumulativeRevenueA['date'], cumulativeRevenueA['revenue']/cumulativeRevenueA['orders'], label='A')
plt.plot(cumulativeRevenueB['date'], cumulativeRevenueB['revenue']/cumulativeRevenueB['orders'], label='B')
plt.xticks(rotation=90)
plt.title('Cumulative average order size by group')
plt.xlabel('Date')
plt.ylabel('average order size')
plt.legend()


Here we can see, again, that between the days 2019-08-17 and 21 the order size for group B spiked; however, then we see a downward trend. It is partigularly interesting since, as we know from the previous graph, the revenue for group B kept steadily increasing. Later we will see if anything changes after we remove abnormalities. 

Graph the relative difference in cumulative average order size for group B
compared with group A

In [ ]:
# gathering the data into one DataFrame
mergedCumulativeRevenue = cumulativeRevenueA.merge(cumulativeRevenueB, left_on='date', right_on='date', how='left', suffixes=['A', 'B'])

# plotting a relative difference graph for the average purchase sizes
plt.plot(mergedCumulativeRevenue['date'], (mergedCumulativeRevenue['revenueB']/mergedCumulativeRevenue['ordersB'])/(mergedCumulativeRevenue['revenueA']/mergedCumulativeRevenue['ordersA'])-1)

# adding the X axis
plt.axhline(y=0, color='black', linestyle='--') 
plt.xticks(rotation=90)
plt.title('Difference in cumulative average order size')
plt.xlabel('Date')
plt.ylabel('Relative difference')

We can see severe fluctuations, serious and sudden changes. It may be connected to abnormally big orders. 

Calculate each group's conversion rate as the ratio of orders to the
number of visits for each day. Plot the daily conversion rates of the two
groups and describe the difference.

In [ ]:
cumulativeData['conversion'] = cumulativeData['orders']/cumulativeData['visitors']

# selecting data on group A 
cumulativeDataA = cumulativeData[cumulativeData['group']=='A']

# selecting data on group B
cumulativeDataB = cumulativeData[cumulativeData['group']=='B']

# plotting the graphs
plt.plot(cumulativeDataA['date'], cumulativeDataA['conversion'], label='A')
plt.plot(cumulativeDataB['date'], cumulativeDataB['conversion'], label='B')
plt.title('Daily conversion rates')
plt.xlabel('Date')
plt.ylabel('Conversion rate')
plt.legend()
plt.xticks(rotation=90)

We can see that in the beginning there were fluctuations and group A started out in better condition than group B. However, after somewhere between 08.05 and 08.09 group B conversion became higher and the trend stabilised: conversion for group B is relatively stable with small spikes, but conversion for group A steadily declines. 

Plotting relative difference in conversion rates:

In [ ]:
mergedCumulativeConversions = cumulativeDataA[['date','conversion']].merge(cumulativeDataB[['date','conversion']], left_on='date', right_on='date', how='left', suffixes=['A', 'B'])

plt.plot(mergedCumulativeConversions['date'], mergedCumulativeConversions['conversionB']/mergedCumulativeConversions['conversionA']-1, label="Relative gain in conversion in group B as opposed to group A")
plt.legend()
plt.xticks(rotation=90)
plt.axhline(y=0, color='black', linestyle='--')
plt.axhline(y=0.2, color='grey', linestyle='--')
plt.title('Relative difference in conversion rates')
plt.xlabel('Date')
plt.ylabel('Difference')

The conversion ratio is not yet stable; however, we can see on this graph that B started low, but quickly went up, and is still fluctuating, alsthough not as severely as in the beginning.

Plot a scatter chart of the number of orders per user. 

In [ ]:
ordersByUsers = orders.drop(['group', 'revenue', 'date'], axis=1).groupby('visitorId', as_index=False).agg({'transactionId' : pd.Series.nunique})
ordersByUsers.columns = ['visitorId','orders']


# the range of numbers from 0 to the number of observations in ordersByUsers
x_values = pd.Series(range(0,len(ordersByUsers)))
plt.scatter(x_values, ordersByUsers['orders']) 

We can see that the vast majority of users make one order, there is a bunch of users with two orders and a minority of users have 3. 3 may be an anomaly. 

Calculate the 95th and 99th percentiles for the number of orders per user.
Define the point at which a data point becomes an anomaly.


In [ ]:
percentiles = np.percentile(ordersByUsers['orders'], [95, 99])
print(percentiles)

Only 5% of users made 2 orders, and only one % made 3 orders. 

From the scatter plot I'd say the abnormal plot is 2.

Histogram and scatter chart of order prices.

In [ ]:
plt.hist(orders['revenue']) 

In [ ]:
x_values = pd.Series(range(0,len(orders['revenue'])))
plt.scatter(x_values, orders['revenue']) 

A vast majority of prices is below 2500$. 

95th and 99th percentiles of order prices.

In [ ]:
print(np.percentile(orders['revenue'], [95, 99])) 

Only 5% of order prices are  more than 415$, and only one more than 830$. 

I'd say the abnormal plot is 830.3.

Calculating the statistical significance of the difference in conversion between the
groups using the raw data.

For each statistical test below I will be using alpha balue of 0.5. I will specify hypotheses as it goes.
For the test below the H0 is that there's not a statistically significant difference in conversion between the groups. H1 is that there is.

In [ ]:
ordersByUsersA = orders[orders['group']=='A'].groupby('visitorId', as_index=False).agg({'transactionId' : pd.Series.nunique})
ordersByUsersA.columns = ['visitorId', 'orders']

ordersByUsersB = orders[orders['group']=='B'].groupby('visitorId', as_index=False).agg({'transactionId' : pd.Series.nunique})
ordersByUsersB.columns = ['visitorIdd', 'orders']

#pd.Series(0, index=np.arange(data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])), name='orders')
    
[ordersByUsersA['orders'],pd.Series(0, index=np.arange(visits[visits['group']=='A']['visits'].sum() - len(ordersByUsersA['orders'])), name='orders')] 

sampleA = pd.concat([ordersByUsersA['orders'],pd.Series(0, index=np.arange(visits[visits['group']=='A']['visits'].sum() - len(ordersByUsersA['orders'])), name='orders')],axis=0)

sampleB = pd.concat([ordersByUsersB['orders'],pd.Series(0, index=np.arange(visits[visits['group']=='B']['visits'].sum() - len(ordersByUsersB['orders'])), name='orders')],axis=0)  
    
    
ordersByUsersA = orders[orders['group']=='A'].groupby('visitorId', as_index=False).agg({'transactionId' : pd.Series.nunique})
ordersByUsersA.columns = ['visitorId', 'orders']

ordersByUsersB = orders[orders['group']=='B'].groupby('visitorId', as_index=False).agg({'transactionId' : pd.Series.nunique})
ordersByUsersB.columns = ['visitorId', 'orders']

sampleA = pd.concat([ordersByUsersA['orders'],pd.Series(0, index=np.arange(visits[visits['group']=='A']['visits'].sum() - len(ordersByUsersA['orders'])), name='orders')],axis=0)

sampleB = pd.concat([ordersByUsersB['orders'],pd.Series(0, index=np.arange(visits[visits['group']=='B']['visits'].sum() - len(ordersByUsersB['orders'])), name='orders')],axis=0)

#relative difference in conversion between the groups: 
print("{0:.5f}".format(st.mannwhitneyu(sampleA, sampleB)[1]))
print("{0:.3f}".format(sampleB.mean()/sampleA.mean()-1)) 

p-value is much lower than 0.05 and the relative conversion gain for group B compared to group A is 16%. Thus we have to reject the H0.

Calculating the statistical significance of the difference in average order size between the groups using the raw data. The H0 is that there is no significant difference in average order size between the groups. H1 is that there is. 

In [ ]:
print("{0:.3f}".format(st.mannwhitneyu(orders[orders['group']=='A']['revenue'], orders[orders['group']=='B']['revenue'])[1]))
print("{0:.3f}".format(orders[orders['group']=='B']['revenue'].mean()/orders[orders['group']=='A']['revenue'].mean()-1)) 

p-value is higher than 0.05, so we can't regect the H0: the hypothesis that there is no serious statistical difference betwen group's average order sizes. Now we will repeat the test after removing anomalies.

Removing anomalies from our data.
95th and 99th percentiles for average order size are 435.54 and 900.904 and 95th and 99th % of orders per user are two and four orders. I decided to define anomalies as more than 2 orders that cost more than 1000 $. Now I will remove them. 

In [ ]:
ordersByUsersA = orders[orders['group']=='A'].groupby('visitorId', as_index=False).agg({'transactionId' : pd.Series.nunique})
ordersByUsersA.columns = ['visitorId', 'orders']

ordersByUsersB = orders[orders['group']=='B'].groupby('visitorId', as_index=False).agg({'transactionId' : pd.Series.nunique})
ordersByUsersB.columns = ['visitorId', 'orders']

usersWithManyOrders = pd.concat([ordersByUsersA[ordersByUsersA['orders'] > 2]['visitorId'], ordersByUsersB[ordersByUsersB['orders'] > 2]['visitorId']], axis = 0)
usersWithExpensiveOrders = orders[orders['revenue'] > 1000]['visitorId']
abnormalUsers = pd.concat([usersWithManyOrders, usersWithExpensiveOrders], axis = 0).drop_duplicates().sort_values(ascending=True)

Now I calculate the difference in conversion after removing the anomalies using the filtered data. H0 is that there's not a statistically significant difference in conversion between the groups. H1 is that there is.

In [ ]:
sampleAFiltered = pd.concat([ordersByUsersA[np.logical_not(ordersByUsersA['visitorId'].isin(abnormalUsers))]['orders'],pd.Series(0, index=np.arange(visits[visits['group']=='A']['visits'].sum() - len(ordersByUsersA['orders'])),name='orders')],axis=0)

sampleBFiltered = pd.concat([ordersByUsersB[np.logical_not(ordersByUsersB['visitorId'].isin(abnormalUsers))]['orders'],pd.Series(0, index=np.arange(visits[visits['group']=='B']['visits'].sum() - len(ordersByUsersB['orders'])),name='orders')],axis=0) 

print("{0:.5f}".format(st.mannwhitneyu(sampleAFiltered, sampleBFiltered)[1]))
print("{0:.3f}".format(sampleBFiltered.mean()/sampleAFiltered.mean()-1)) 

p-value is lower than 0.05, so we have to reject H0 hypotheses that there is no statistically important difference in conversion for two groups. The statistical difference between groups is about 18%, which is very high. Group B is in the lead. 

Removing the anomalies and calculating the statistical significance for the difference in average order size using the filtered data. The H0 is that there is no significant difference in average order size between the groups. H1 is that there is. 

In [ ]:
print("{0:.3f}".format(st.mannwhitneyu(
    orders[np.logical_and(
        orders['group']=='A',
        np.logical_not(orders['visitorId'].isin(abnormalUsers)))]['revenue'],
    orders[np.logical_and(
        orders['group']=='B',
        np.logical_not(orders['visitorId'].isin(abnormalUsers)))]['revenue'])[1]))

print("{0:.3f}".format(
    orders[np.logical_and(orders['group']=='B',np.logical_not(orders['visitorId'].isin(abnormalUsers)))]['revenue'].mean()/
    orders[np.logical_and(
        orders['group']=='A',
        np.logical_not(orders['visitorId'].isin(abnormalUsers)))]['revenue'].mean() - 1)) 

P-value is higher than 0.05,so we can not reject the H0. The difference in order size is small,about 3.5%. In group A it is actually higher, but it doesn't change the difference in revenue. 

The overall conclusion:

There's a statistically significant difference in conversion between the groups, according to both the raw and the filtered data.
The raw data didn't show a statistically significant difference between the groups in terms of average purchase size even after the anomalies were removed.
The graph of the difference in conversion between the groups shows that group B's results are better than those of group A.
The graph of the difference in average purchase size shows fluctuations; however, there was no statistical significance to that difference.

Based on the facts listed above, I think we should conclude the test: group B has done better overal when it came to revenue and conversion and there was no serious statistical difference between order sizes, even after the removal of anomalies. 